In [1]:
import os
import shutil
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from PIL import Image
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.optim as optim
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
!git clone https://github.com/mazurowski-lab/finetune-SAM.git

Cloning into 'finetune-SAM'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 408 (delta 21), reused 22 (delta 11), pack-reused 369 (from 1)
Receiving objects: 100% (408/408), 1.63 MiB | 29.21 MiB/s, done.
Resolving deltas: 100% (148/148), done.


In [3]:
def ensure_dir(directory):
    """Ensure output directory exists for saving images and masks,if not create it."""
    if not os.path.exists(directory):
        os.makedirs(directory)

In [4]:
def prepare_output_dirs(output_dir):
    """Create and return output directories for images and masks."""
    output_images_dir = os.path.join(output_dir, 'images')
    output_masks_dir = os.path.join(output_dir, 'masks')

    # Ensure output directories exist
    ensure_dir(output_images_dir)
    ensure_dir(output_masks_dir)

    return output_images_dir, output_masks_dir

In [5]:
def process_subjects(input_dir, output_dir, category):
    """Process subject folders, rename files, and copy to output directory."""
    # Sort subject folders for consistent processing order
    subject_folders = sorted(os.listdir(input_dir))
    
    print("--------------------------------------------------")
    print(f"Starting processing for category: {category}")
    print("--------------------------------------------------")
    
    for subject_folder in subject_folders:
        subject_path = os.path.join(input_dir, subject_folder)
        if not os.path.isdir(subject_path):
            continue

        subject_id = subject_folder.split('_')[-1]  # Extract ID, e.g., "0" from "Subject_0"
        print("--------------------------------------------------")
        print(f"Processing Subject: {subject_folder} (ID: {subject_id}) - Category: {category}")
        print("--------------------------------------------------")

        # Get all files in the subject folder and sort them for consistency
        file_list = sorted([file_name for file_name in os.listdir(subject_path) if os.path.isfile(os.path.join(subject_path, file_name))])
        print(f"Number of files in {subject_folder} ({category}): {len(file_list)}")

        for file_name in file_list:
            file_path = os.path.join(subject_path, file_name)
            if category=='masks':
                # Rename the file to subjectid_imagenumber
                new_file_name = f"mask_{subject_id}_{file_name}"
                output_path = os.path.join(output_dir, new_file_name)
            else:
                new_file_name = f"{subject_id}_{file_name}"
                output_path = os.path.join(output_dir, new_file_name)
                
                # Copy file to output directory
            shutil.copy(file_path, output_path)
            # print(f"Saved: {output_path}")

    print("--------------------------------------------------")
    print(f"Finished processing for category: {category}")
    print("--------------------------------------------------")

In [6]:
def process_folder(data_dir, output_dir):
    """Flatten folder structure, rename files, and save paths."""
    print("==================================================")
    print(f"Processing folder: {data_dir}")
    print("==================================================")

    # Prepare output directories
    output_images_dir, output_masks_dir = prepare_output_dirs(output_dir)

    for category in ['images', 'masks']:
        input_dir = os.path.join(data_dir, category)
        output_category_dir = output_images_dir if category == 'images' else output_masks_dir

        # Process subjects in the input directory
        process_subjects(input_dir, output_category_dir, category)

    print("==================================================")
    print(f"Finished processing folder: {data_dir}")
    print("==================================================")

In [7]:
def main():
    base_dir = "/kaggle/input/computer-vision-project"  
    output_dir = "/kaggle/working/finetune-SAM"  

    # Process train and validation folders
    for folder in ['train', 'val']:
        data_dir = os.path.join(base_dir, folder)
        output_folder = os.path.join(output_dir, folder)
        process_folder(data_dir, output_folder)

In [8]:
main()

Processing folder: /kaggle/input/computer-vision-project/train
--------------------------------------------------
Starting processing for category: images
--------------------------------------------------
--------------------------------------------------
Processing Subject: Subject_0 (ID: 0) - Category: images
--------------------------------------------------
Number of files in Subject_0 (images): 75
--------------------------------------------------
Processing Subject: Subject_1 (ID: 1) - Category: images
--------------------------------------------------
Number of files in Subject_1 (images): 26
--------------------------------------------------
Processing Subject: Subject_10 (ID: 10) - Category: images
--------------------------------------------------
Number of files in Subject_10 (images): 18
--------------------------------------------------
Processing Subject: Subject_11 (ID: 11) - Category: images
--------------------------------------------------
Number of files in Subject_

In [9]:
import os
import pandas as pd

# Define paths for training images and masks
image_folder_train = "/kaggle/working/finetune-SAM/train/images"
mask_folder_train = "/kaggle/working/finetune-SAM/train/masks"

# Get sorted list of image and mask files
image_files_train = sorted(os.listdir(image_folder_train))
mask_files_train = sorted(os.listdir(mask_folder_train))

# Ensure that the number of image files matches the number of mask files
assert len(image_files_train) == len(mask_files_train), "Mismatch between number of images and masks in the training set!"

# Create train data dictionary with image and mask paths
train_data = {
    image_files_train[0]: [img for img in image_files_train],
    mask_files_train[0]: [mask for mask in mask_files_train],
}

# Create a DataFrame and save it as CSV
df_train = pd.DataFrame(train_data)
df_train.to_csv('/kaggle/working/finetune-SAM/train/train_5shot.csv', index=False)

# Define paths for validation images and masks
image_folder_val = "/kaggle/working/finetune-SAM/val/images"
mask_folder_val = "/kaggle/working/finetune-SAM/val/masks"

# Get sorted list of image and mask files for validation
image_files_val = sorted(os.listdir(image_folder_val))
mask_files_val = sorted(os.listdir(mask_folder_val))

# Ensure that the number of image files matches the number of mask files
assert len(image_files_val) == len(mask_files_val), "Mismatch between number of images and masks in the validation set!"

# Create validation data dictionary with image and mask paths
val_data = {
    image_files_val[0]: [img for img in image_files_val],
    mask_files_val[0]: [mask for mask in mask_files_val],
}

# Create a DataFrame and save it as CSV
df_val = pd.DataFrame(val_data)
df_val.to_csv('/kaggle/working/finetune-SAM/val/val_5shot.csv', index=False)

print("CSV files created successfully!")

CSV files created successfully!


In [10]:
df_train.head()

,0_0.png,mask_0_0.png
0,0_0.png,mask_0_0.png
1,0_1.png,mask_0_1.png
2,0_10.png,mask_0_10.png
3,0_100.png,mask_0_100.png
4,0_101.png,mask_0_101.png


In [11]:
with open("/kaggle/working/finetune-SAM/train/train_5shot.csv",'r') as f:
    line = f.readline()
    print(line)

0_0.png,mask_0_0.png



In [12]:
%cd /kaggle/working/finetune-SAM

/kaggle/working/finetune-SAM


In [13]:
os.makedirs("Dataset",exist_ok=True)
os.makedirs("Dataset/Lung-Tumor",exist_ok=True)
os.makedirs("Dataset/Lung-Tumor/images",exist_ok=True)
import shutil
import os

# Specify source and destination folders
source_folder = '/kaggle/working/finetune-SAM/train/images'
destination_folder = '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/images'

# List all files in the source folder
files = os.listdir(source_folder)

# Move all files from source to destination folder
for file in files:
    shutil.move(os.path.join(source_folder, file), destination_folder)

os.makedirs("Dataset/Lung-Tumor/masks",exist_ok=True)
source_folder = '/kaggle/working/finetune-SAM/train/masks'
destination_folder = '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/masks'

# List all files in the source folder
files = os.listdir(source_folder)

# Move all files from source to destination folder
for file in files:
    shutil.move(os.path.join(source_folder, file), destination_folder)

In [14]:
source_folder = '/kaggle/working/finetune-SAM/val/images'
destination_folder = '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/images'

# List all files in the source folder
files = os.listdir(source_folder)

# Move all files from source to destination folder
for file in files:
    shutil.move(os.path.join(source_folder, file), destination_folder)

os.makedirs("Dataset/Lung-Tumor/masks",exist_ok=True)
source_folder = '/kaggle/working/finetune-SAM/val/masks'
destination_folder = '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/masks'

# List all files in the source folder
files = os.listdir(source_folder)

# Move all files from source to destination folder
for file in files:
    shutil.move(os.path.join(source_folder, file), destination_folder)

In [15]:
shutil.move("/kaggle/working/finetune-SAM/train/train_5shot.csv", '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor')
shutil.move("/kaggle/working/finetune-SAM/val/val_5shot.csv", '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor')

'/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/val_5shot.csv'

In [16]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 57.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 2.7 MB/s eta 0:00:00
Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation 
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 100.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.3/

In [17]:
os.makedirs("Model",exist_ok=True)
model_dir = "/kaggle/input/med_sam/pytorch/default/1/medsam_vit_b.pth"  # Path to the model directory
output_dir = "/kaggle/working/finetune-SAM/Model"
shutil.copy(model_dir, output_dir)

'/kaggle/working/finetune-SAM/Model/medsam_vit_b.pth'

In [18]:
import os
import subprocess

# Set up environment variables
arch = "vit_b"  # Choose architecture
finetune_type = "adapter"  # Finetune type
dataset_name = "/Lung-Tumor"  # Adjust this based on your dataset
targets = 'combine_all'  # Set for binary or multi-class segmentation

# Construct the paths to train and validation datasets
img_folder = "/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/images"  # Folder where images are stored
mask_folder = "/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/masks"  # Folder where images are stored
train_img_list = "/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/train_5shot.csv"
val_img_list = "/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/val_5shot.csv"

# Set checkpoint directory for saving the model
dir_checkpoint = f"/kaggle/working/finetune-SAM/Model"

# Construct the command for running the training
command = [
    "python", "SingleGPU_train_finetune_noprompt.py",
    "-if_warmup", "True",
    "-finetune_type", finetune_type,
    "-arch", arch,
    "-if_mask_decoder_adapter", "True",
    "-if_update_encoder","True",
    "-if_mask_decoder_adapter","True",
    "-if_encoder_adapter","True",
    "-normalize_type", 'medsam',
    '-warmup_period',"20",
    '-num_cls','2',
    "-epochs","100",
    "-img_folder", img_folder,
    "-mask_folder", mask_folder,
    "-sam_ckpt", "/kaggle/working/finetune-SAM/Model/medsam_vit_b.pth",  # You may need to upload the checkpoint
    "-dataset_name", dataset_name,
    "-dir_checkpoint", dir_checkpoint,
    "-train_img_list", train_img_list,
    "-val_img_list", val_img_list,
    "-b",'4'
]

# Run the training command
subprocess.run(command)

/kaggle/working/finetune-SAM/models/sam/modeling/tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/kaggle/working/finetune-SAM/models/sam/modeling/tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_11m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/kaggle/working/finetune-SAM/models/sam/modeling/tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_21m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/kaggle/working

train dataset: /Lung-Tumor
combine_all
Filtered data list to 1547 entries.
Filtered data list to 78 entries.
if update encoder: True
if image encoder adapter: False
if mask decoder adapter: True


  0%|          | 0/387 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass

100%|██████████| 387/387 [06:50<00:00,  1.06s/it]
Epoch num 0| train loss 0.5266975909086469 
100%|██████████| 20/20 [00:21<00:00,  1.07s/it]
Epoch num 0| train loss 0.5266975909086469 
  0%|          | 0/387 [00:00<?, ?it/s]

Eval Epoch num 0 | val loss 0.4986049309372902 | dsc 0.5164740085601807 

largest DSC now: 0.5164740085601807



100%|██████████| 387/387 [06:48<00:00,  1.06s/it]
Epoch num 1| train loss 0.48297848612147093 
Epoch num 1| train loss 0.48297848612147093 2.53s/it]
100%|██████████| 387/387 [06:49<00:00,  1.06s/it]
Epoch num 2| train loss 0.4620461165134913 
100%|██████████| 20/20 [00:21<00:00,  1.07s/it]
Epoch num 2| train loss 0.4620461165134913 
  0%|          | 0/387 [00:00<?, ?it/s]

...


100%|██████████| 387/387 [06:48<00:00,  1.06s/it]
Epoch num 61| train loss 0.3091945720966472 
Epoch num 61| train loss 0.3091945720966472 421.19s/it] 
100%|██████████| 387/387 [06:48<00:00,  1.06s/it]
Epoch num 62| train loss 0.30618369026778586 
100%|██████████| 20/20 [00:21<00:00,  1.07s/it]
Epoch num 62| train loss 0.30618369026778586 
  0%|          | 0/387 [00:00<?, ?it/s]

Eval Epoch num 62 | val loss 0.2440075319260359 | dsc 0.7061777114868164 




100%|██████████| 387/387 [06:48<00:00,  1.05s/it]
Epoch num 63| train loss 0.30350839564196513 
Epoch num 63| train loss 0.30350839564196513 21.10s/it]
100%|██████████| 387/387 [06:47<00:00,  1.05s/it]
Epoch num 64| train loss 0.3044116168213137 
100%|██████████| 20/20 [00:21<00:00,  1.08s/it]
Epoch num 64| train loss 0.3044116168213137 
:  64%|██████▍   | 64/100 [7:34:06<4:15:26, 425.72s/it]


Eval Epoch num 64 | val loss 0.24312278404831886 | dsc 0.7094900012016296 

Training finished###########


CompletedProcess(args=['python', 'SingleGPU_train_finetune_noprompt.py', '-if_warmup', 'True', '-finetune_type', 'adapter', '-arch', 'vit_b', '-if_mask_decoder_adapter', 'True', '-if_update_encoder', 'True', '-if_mask_decoder_adapter', 'True', '-normalize_type', 'medsam', '-warmup_period', '20', '-num_cls', '2', '-epochs', '100', '-img_folder', '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/images', '-mask_folder', '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/masks', '-sam_ckpt', '/kaggle/working/finetune-SAM/Model/medsam_vit_b.pth', '-dataset_name', '/Lung-Tumor', '-dir_checkpoint', '/kaggle/working/finetune-SAM/Model', '-train_img_list', '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/train_5shot.csv', '-val_img_list', '/kaggle/working/finetune-SAM/Dataset/Lung-Tumor/val_5shot.csv', '-b', '4'], returncode=0)